# 🚀 Chat Analytics Dashboard

**Interactive Visualization of AI Development Conversations**

---

A premium, interactive exploration of 31 chat sessions containing 483 messages spanning 6 days of intensive AI-assisted development.

> 💡 **Tip**: Hover over charts for detailed information. Use the toolbar to zoom, pan, and export.

In [2]:
# === SETUP: Import Libraries & Configure Theme ===
import sqlite3
import json
from collections import Counter, defaultdict
from datetime import datetime
from pathlib import Path

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

# Set premium dark theme
pio.templates.default = "plotly_dark"

# Custom color palette - Neon Cyberpunk
COLORS = {
    'primary': '#00D9FF',      # Cyan
    'secondary': '#FF006E',    # Magenta
    'accent': '#8338EC',       # Purple
    'success': '#00F5D4',      # Teal
    'warning': '#FFD60A',      # Yellow
    'danger': '#FF5E5B',       # Red
    'background': '#0D1117',   # Deep dark
    'surface': '#161B22',      # Card background
    'text': '#E6EDF3',         # Light text
}

# Gradient color scales
NEON_GRADIENT = [
    [0, '#00D9FF'],
    [0.25, '#8338EC'],
    [0.5, '#FF006E'],
    [0.75, '#FFD60A'],
    [1, '#00F5D4']
]

print("✨ Dashboard initialized with Cyberpunk Neon theme")

✨ Dashboard initialized with Cyberpunk Neon theme


In [3]:
# === DATA LOADING: Fetch from Knowledge Database ===

def load_chat_data(db_path="../.workspace/knowledge.db"):
    """Load all chat data from the knowledge database."""
    conn = sqlite3.connect(db_path)
    
    # Sessions DataFrame
    sessions_df = pd.read_sql_query("""
        SELECT 
            id as session_id,
            title,
            message_count,
            user_message_count,
            assistant_message_count,
            first_message_at,
            last_message_at
        FROM chat_sessions
        ORDER BY message_count DESC
    """, conn)
    
    # Messages DataFrame
    messages_df = pd.read_sql_query("""
        SELECT 
            session_id,
            role,
            LENGTH(content) as content_length,
            sequence_num
        FROM chat_messages
        ORDER BY session_id, sequence_num
    """, conn)
    
    # Source files DataFrame
    sources_df = pd.read_sql_query("""
        SELECT 
            file_path,
            message_count,
            new_messages,
            imported_at
        FROM chat_source_files
    """, conn)
    
    conn.close()
    
    # Add derived columns
    sessions_df['grade'] = sessions_df['message_count'].apply(assign_grade)
    sessions_df['grade_color'] = sessions_df['grade'].map({
        'A': COLORS['success'],
        'B': COLORS['primary'],
        'C': COLORS['warning'],
        'D': COLORS['secondary'],
        'F': COLORS['danger']
    })
    
    return sessions_df, messages_df, sources_df

def assign_grade(msg_count):
    """Assign quality grade based on message count."""
    if msg_count >= 40: return 'A'
    elif msg_count >= 20: return 'B'
    elif msg_count >= 10: return 'C'
    elif msg_count >= 5: return 'D'
    else: return 'F'

# Load the data
sessions_df, messages_df, sources_df = load_chat_data()

print(f"📊 Loaded {len(sessions_df)} sessions with {len(messages_df)} messages")
print(f"📁 From {len(sources_df)} source files")

📊 Loaded 31 sessions with 483 messages
📁 From 40 source files


---

## 📈 Key Performance Indicators

In [4]:
# === KPI CARDS: Animated Number Display ===

total_sessions = len(sessions_df)
total_messages = sessions_df['message_count'].sum()
avg_per_session = total_messages / total_sessions
total_content_kb = messages_df['content_length'].sum() / 1024

# Create indicator cards
fig_kpi = make_subplots(
    rows=1, cols=4,
    specs=[[{'type': 'indicator'}] * 4],
    horizontal_spacing=0.05
)

# Sessions indicator
fig_kpi.add_trace(go.Indicator(
    mode="number+delta",
    value=total_sessions,
    title={"text": "<b>Sessions</b><br><span style='font-size:0.8em;color:gray'>Total Conversations</span>"},
    number={'font': {'size': 60, 'color': COLORS['primary']}},
    delta={'reference': 25, 'relative': True, 'valueformat': '.0%'},
), row=1, col=1)

# Messages indicator
fig_kpi.add_trace(go.Indicator(
    mode="number",
    value=total_messages,
    title={"text": "<b>Messages</b><br><span style='font-size:0.8em;color:gray'>User Inputs</span>"},
    number={'font': {'size': 60, 'color': COLORS['success']}},
), row=1, col=2)

# Average indicator
fig_kpi.add_trace(go.Indicator(
    mode="number+gauge",
    value=avg_per_session,
    title={"text": "<b>Avg/Session</b><br><span style='font-size:0.8em;color:gray'>Messages</span>"},
    number={'font': {'size': 50, 'color': COLORS['accent']}, 'suffix': ''},
    gauge={
        'axis': {'range': [0, 30], 'tickwidth': 0},
        'bar': {'color': COLORS['accent']},
        'bgcolor': COLORS['surface'],
        'borderwidth': 0,
        'steps': [
            {'range': [0, 10], 'color': 'rgba(255,94,91,0.3)'},
            {'range': [10, 20], 'color': 'rgba(255,214,10,0.3)'},
            {'range': [20, 30], 'color': 'rgba(0,245,212,0.3)'}
        ],
    }
), row=1, col=3)

# Content size indicator
fig_kpi.add_trace(go.Indicator(
    mode="number",
    value=total_content_kb,
    title={"text": "<b>Content</b><br><span style='font-size:0.8em;color:gray'>Total KB</span>"},
    number={'font': {'size': 60, 'color': COLORS['warning']}, 'suffix': ' KB', 'valueformat': ',.0f'},
), row=1, col=4)

fig_kpi.update_layout(
    height=250,
    paper_bgcolor=COLORS['background'],
    plot_bgcolor=COLORS['background'],
    margin=dict(l=20, r=20, t=60, b=20),
    title={
        'text': '🎯 Key Performance Indicators',
        'font': {'size': 24, 'color': COLORS['text']},
        'x': 0.5
    }
)

fig_kpi.show()

---

## 🎨 Session Quality Distribution

In [5]:
# === SUNBURST CHART: Hierarchical Grade Distribution ===

# Prepare hierarchical data
grade_counts = sessions_df['grade'].value_counts().to_dict()

# Build sunburst data
sunburst_data = []
for grade in ['A', 'B', 'C', 'D', 'F']:
    count = grade_counts.get(grade, 0)
    if count > 0:
        sessions_in_grade = sessions_df[sessions_df['grade'] == grade]
        for _, row in sessions_in_grade.iterrows():
            sunburst_data.append({
                'grade': f"Grade {grade}",
                'session': row['title'][:25],
                'messages': row['message_count'],
                'full_title': row['title']
            })

sunburst_df = pd.DataFrame(sunburst_data)

fig_sunburst = px.sunburst(
    sunburst_df,
    path=['grade', 'session'],
    values='messages',
    color='messages',
    color_continuous_scale=[
        [0, COLORS['danger']],
        [0.3, COLORS['warning']],
        [0.6, COLORS['primary']],
        [1, COLORS['success']]
    ],
    hover_data=['full_title']
)

fig_sunburst.update_layout(
    title={
        'text': '🌟 Session Quality Hierarchy',
        'font': {'size': 24, 'color': COLORS['text']},
        'x': 0.5
    },
    paper_bgcolor=COLORS['background'],
    height=600,
    margin=dict(l=20, r=20, t=80, b=20),
)

fig_sunburst.update_traces(
    textinfo='label+percent entry',
    insidetextorientation='radial',
    marker=dict(line=dict(color=COLORS['background'], width=2))
)

fig_sunburst.show()

In [6]:
# === TREEMAP: Visual Size Representation ===

fig_treemap = px.treemap(
    sessions_df,
    path=[px.Constant('All Sessions'), 'grade', 'title'],
    values='message_count',
    color='message_count',
    color_continuous_scale=[
        [0, '#1a1a2e'],
        [0.2, COLORS['danger']],
        [0.4, COLORS['secondary']],
        [0.6, COLORS['warning']],
        [0.8, COLORS['primary']],
        [1, COLORS['success']]
    ],
    hover_data=['session_id', 'message_count']
)

fig_treemap.update_layout(
    title={
        'text': '📦 Session Size Treemap',
        'font': {'size': 24, 'color': COLORS['text']},
        'x': 0.5
    },
    paper_bgcolor=COLORS['background'],
    height=600,
    margin=dict(l=10, r=10, t=80, b=10),
)

fig_treemap.update_traces(
    textinfo='label+value',
    textfont={'size': 14},
    marker=dict(line=dict(color=COLORS['background'], width=2))
)

fig_treemap.show()

---

## 📊 Session Rankings

In [7]:
# === HORIZONTAL BAR RACE: Top Sessions with Animation ===

top_15 = sessions_df.head(15).copy()
top_15['title_short'] = top_15['title'].str[:30]

fig_bars = go.Figure()

# Add bars with gradient effect
fig_bars.add_trace(go.Bar(
    y=top_15['title_short'][::-1],
    x=top_15['message_count'][::-1],
    orientation='h',
    marker=dict(
        color=top_15['message_count'][::-1],
        colorscale=[
            [0, COLORS['secondary']],
            [0.5, COLORS['accent']],
            [1, COLORS['primary']]
        ],
        line=dict(color=COLORS['text'], width=1)
    ),
    text=top_15['message_count'][::-1],
    textposition='outside',
    textfont=dict(color=COLORS['text'], size=12),
    hovertemplate='<b>%{y}</b><br>Messages: %{x}<extra></extra>'
))

# Add grade badges
for i, (_, row) in enumerate(top_15[::-1].iterrows()):
    fig_bars.add_annotation(
        x=row['message_count'] + 8,
        y=i,
        text=f"<b>{row['grade']}</b>",
        showarrow=False,
        font=dict(size=14, color=row['grade_color']),
        bgcolor=COLORS['surface'],
        bordercolor=row['grade_color'],
        borderwidth=2,
        borderpad=4
    )

fig_bars.update_layout(
    title={
        'text': '🏆 Top 15 Sessions by Message Count',
        'font': {'size': 24, 'color': COLORS['text']},
        'x': 0.5
    },
    xaxis=dict(
        title='Number of Messages',
        gridcolor='rgba(255,255,255,0.1)',
        showgrid=True,
        range=[0, max(top_15['message_count']) + 20]
    ),
    yaxis=dict(
        title='',
        gridcolor='rgba(255,255,255,0.05)',
    ),
    paper_bgcolor=COLORS['background'],
    plot_bgcolor=COLORS['background'],
    height=600,
    margin=dict(l=200, r=80, t=80, b=60),
    showlegend=False
)

fig_bars.show()

---

## 🔥 Topic Analysis

In [8]:
# === WORD CLOUD ALTERNATIVE: Bubble Chart of Keywords ===

# Extract keywords from titles
all_words = []
for title in sessions_df['title']:
    words = title.lower().replace('-', ' ').replace('_', ' ').split()
    all_words.extend([w for w in words if len(w) > 3])

word_counts = Counter(all_words).most_common(20)
keywords_df = pd.DataFrame(word_counts, columns=['keyword', 'count'])
keywords_df['size'] = keywords_df['count'] * 15  # Scale for visibility

# Create a packed bubble-like scatter
np.random.seed(42)
keywords_df['x'] = np.random.uniform(0, 10, len(keywords_df))
keywords_df['y'] = np.random.uniform(0, 10, len(keywords_df))

fig_bubble = go.Figure()

fig_bubble.add_trace(go.Scatter(
    x=keywords_df['x'],
    y=keywords_df['y'],
    mode='markers+text',
    marker=dict(
        size=keywords_df['size'],
        color=keywords_df['count'],
        colorscale=[
            [0, 'rgba(131,56,236,0.6)'],
            [0.5, 'rgba(0,217,255,0.8)'],
            [1, 'rgba(0,245,212,1)']
        ],
        line=dict(color=COLORS['text'], width=1),
        sizemode='area',
        sizeref=0.5,
    ),
    text=keywords_df['keyword'],
    textposition='middle center',
    textfont=dict(
        size=keywords_df['count'] * 2 + 8,
        color=COLORS['text']
    ),
    hovertemplate='<b>%{text}</b><br>Frequency: %{marker.color}<extra></extra>'
))

fig_bubble.update_layout(
    title={
        'text': '💬 Topic Keywords Visualization',
        'font': {'size': 24, 'color': COLORS['text']},
        'x': 0.5
    },
    xaxis=dict(visible=False),
    yaxis=dict(visible=False),
    paper_bgcolor=COLORS['background'],
    plot_bgcolor=COLORS['background'],
    height=500,
    margin=dict(l=20, r=20, t=80, b=20),
    showlegend=False
)

fig_bubble.show()

In [9]:
# === RADAR CHART: Topic Category Distribution ===

# Categorize sessions
categories = {
    'Documentation': ['documentation', 'docs', 'spec', 'disc', 'adr'],
    'RAG & Knowledge': ['rag', 'knowledge', 'memory', 'embedding', 'graph'],
    'Workflow': ['workflow', 'plan', 'stage', 'closure'],
    'Architecture': ['architecture', 'design', 'adapter', 'refactor'],
    'Infrastructure': ['docker', 'python', 'environment', 'migration'],
    'Quality': ['compliance', 'validation', 'drift', 'audit']
}

category_counts = {cat: 0 for cat in categories}
for title in sessions_df['title']:
    title_lower = title.lower()
    for cat, keywords in categories.items():
        if any(kw in title_lower for kw in keywords):
            category_counts[cat] += 1

radar_df = pd.DataFrame({
    'category': list(category_counts.keys()),
    'count': list(category_counts.values())
})

fig_radar = go.Figure()

fig_radar.add_trace(go.Scatterpolar(
    r=radar_df['count'].tolist() + [radar_df['count'].iloc[0]],  # Close the shape
    theta=radar_df['category'].tolist() + [radar_df['category'].iloc[0]],
    fill='toself',
    fillcolor='rgba(0,217,255,0.3)',
    line=dict(color=COLORS['primary'], width=3),
    marker=dict(size=10, color=COLORS['success']),
    name='Session Count'
))

fig_radar.update_layout(
    title={
        'text': '🎯 Topic Category Distribution',
        'font': {'size': 24, 'color': COLORS['text']},
        'x': 0.5
    },
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, max(category_counts.values()) + 2],
            gridcolor='rgba(255,255,255,0.2)',
            linecolor='rgba(255,255,255,0.3)'
        ),
        angularaxis=dict(
            gridcolor='rgba(255,255,255,0.2)',
            linecolor='rgba(255,255,255,0.3)'
        ),
        bgcolor=COLORS['surface']
    ),
    paper_bgcolor=COLORS['background'],
    height=500,
    margin=dict(l=80, r=80, t=100, b=60),
    showlegend=False
)

fig_radar.show()

---

## 📉 Distribution Analysis

In [10]:
# === VIOLIN + BOX PLOT: Message Distribution ===

fig_violin = go.Figure()

# Violin plot
fig_violin.add_trace(go.Violin(
    y=sessions_df['message_count'],
    box_visible=True,
    meanline_visible=True,
    fillcolor='rgba(131,56,236,0.5)',
    line_color=COLORS['accent'],
    marker=dict(color=COLORS['primary'], size=8),
    points='all',
    jitter=0.3,
    pointpos=-1.5,
    name='Messages'
))

fig_violin.update_layout(
    title={
        'text': '🎻 Message Count Distribution',
        'font': {'size': 24, 'color': COLORS['text']},
        'x': 0.5
    },
    yaxis=dict(
        title='Messages per Session',
        gridcolor='rgba(255,255,255,0.1)',
        zeroline=False
    ),
    xaxis=dict(visible=False),
    paper_bgcolor=COLORS['background'],
    plot_bgcolor=COLORS['background'],
    height=500,
    margin=dict(l=60, r=60, t=80, b=40),
    showlegend=False
)

# Add mean annotation
mean_val = sessions_df['message_count'].mean()
fig_violin.add_annotation(
    x=0.5, y=mean_val,
    text=f"Mean: {mean_val:.1f}",
    showarrow=True,
    arrowhead=2,
    arrowcolor=COLORS['warning'],
    font=dict(color=COLORS['warning'], size=14),
    ax=100, ay=0
)

fig_violin.show()

In [11]:
# === DONUT CHART: Grade Distribution with Animation ===

grade_data = sessions_df['grade'].value_counts().sort_index()
grade_colors = [COLORS['success'], COLORS['primary'], COLORS['warning'], 
                COLORS['secondary'], COLORS['danger']]

fig_donut = go.Figure()

fig_donut.add_trace(go.Pie(
    labels=[f"Grade {g}" for g in grade_data.index],
    values=grade_data.values,
    hole=0.6,
    marker=dict(
        colors=grade_colors[:len(grade_data)],
        line=dict(color=COLORS['background'], width=3)
    ),
    textinfo='label+percent',
    textposition='outside',
    textfont=dict(color=COLORS['text'], size=14),
    hovertemplate='<b>%{label}</b><br>Sessions: %{value}<br>Percentage: %{percent}<extra></extra>',
    pull=[0.05 if g == 'A' else 0 for g in grade_data.index]  # Pull out Grade A
))

# Center text
fig_donut.add_annotation(
    text=f"<b>{len(sessions_df)}</b><br>Sessions",
    x=0.5, y=0.5,
    font=dict(size=28, color=COLORS['text']),
    showarrow=False
)

fig_donut.update_layout(
    title={
        'text': '🎖️ Quality Grade Distribution',
        'font': {'size': 24, 'color': COLORS['text']},
        'x': 0.5
    },
    paper_bgcolor=COLORS['background'],
    height=500,
    margin=dict(l=20, r=20, t=80, b=20),
    showlegend=True,
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=-0.15,
        xanchor='center',
        x=0.5,
        font=dict(color=COLORS['text'])
    )
)

fig_donut.show()

---

## 📋 Interactive Session Table

In [12]:
# === INTERACTIVE TABLE: Full Session Details ===

table_df = sessions_df[['title', 'message_count', 'grade', 'session_id']].copy()
table_df['rank'] = range(1, len(table_df) + 1)

# Color-coded cells
def grade_to_color(grade):
    return {
        'A': 'rgba(0,245,212,0.3)',
        'B': 'rgba(0,217,255,0.3)',
        'C': 'rgba(255,214,10,0.3)',
        'D': 'rgba(255,0,110,0.3)',
        'F': 'rgba(255,94,91,0.3)'
    }.get(grade, 'rgba(255,255,255,0.1)')

cell_colors = [grade_to_color(g) for g in table_df['grade']]

fig_table = go.Figure(data=[go.Table(
    header=dict(
        values=['<b>Rank</b>', '<b>Session Title</b>', '<b>Messages</b>', '<b>Grade</b>', '<b>Session ID</b>'],
        fill_color=COLORS['surface'],
        font=dict(color=COLORS['text'], size=14),
        align='left',
        height=40,
        line=dict(color=COLORS['primary'], width=2)
    ),
    cells=dict(
        values=[
            table_df['rank'],
            table_df['title'],
            table_df['message_count'],
            table_df['grade'],
            table_df['session_id']
        ],
        fill_color=[cell_colors] * 5,
        font=dict(color=COLORS['text'], size=12),
        align='left',
        height=30,
        line=dict(color='rgba(255,255,255,0.1)', width=1)
    )
)])

fig_table.update_layout(
    title={
        'text': '📑 Complete Session Registry',
        'font': {'size': 24, 'color': COLORS['text']},
        'x': 0.5
    },
    paper_bgcolor=COLORS['background'],
    height=800,
    margin=dict(l=20, r=20, t=80, b=20),
)

fig_table.show()

---

## 🔮 Message Length Analysis

In [13]:
# === HISTOGRAM + RUG: Message Length Distribution ===

msg_lengths = messages_df['content_length'].dropna()

fig_hist = go.Figure()

# Main histogram
fig_hist.add_trace(go.Histogram(
    x=msg_lengths,
    nbinsx=50,
    marker=dict(
        color=msg_lengths[:50],
        colorscale=[
            [0, COLORS['accent']],
            [0.5, COLORS['primary']],
            [1, COLORS['success']]
        ],
        line=dict(color=COLORS['text'], width=0.5)
    ),
    opacity=0.8,
    hovertemplate='Length: %{x}<br>Count: %{y}<extra></extra>'
))

# Add rug plot
fig_hist.add_trace(go.Scatter(
    x=msg_lengths.sample(min(100, len(msg_lengths))),
    y=[-2] * min(100, len(msg_lengths)),
    mode='markers',
    marker=dict(symbol='line-ns', color=COLORS['secondary'], size=10),
    showlegend=False,
    hoverinfo='skip'
))

# Add mean and median lines
mean_len = msg_lengths.mean()
median_len = msg_lengths.median()

fig_hist.add_vline(x=mean_len, line_dash="dash", line_color=COLORS['warning'],
                   annotation_text=f"Mean: {mean_len:,.0f}", annotation_position="top")
fig_hist.add_vline(x=median_len, line_dash="dot", line_color=COLORS['success'],
                   annotation_text=f"Median: {median_len:,.0f}", annotation_position="bottom")

fig_hist.update_layout(
    title={
        'text': '📏 Message Length Distribution',
        'font': {'size': 24, 'color': COLORS['text']},
        'x': 0.5
    },
    xaxis=dict(
        title='Message Length (characters)',
        gridcolor='rgba(255,255,255,0.1)',
        type='log'  # Log scale for better visualization
    ),
    yaxis=dict(
        title='Frequency',
        gridcolor='rgba(255,255,255,0.1)'
    ),
    paper_bgcolor=COLORS['background'],
    plot_bgcolor=COLORS['background'],
    height=500,
    margin=dict(l=60, r=40, t=80, b=60),
    bargap=0.05,
    showlegend=False
)

fig_hist.show()

---

## 🎬 Animated Summary

In [14]:
# === ANIMATED SCATTER: Session Journey ===

# Create animation data
animated_df = sessions_df.copy()
animated_df['cumulative_messages'] = animated_df['message_count'].cumsum()
animated_df['session_num'] = range(1, len(animated_df) + 1)

fig_animated = px.scatter(
    animated_df,
    x='session_num',
    y='message_count',
    size='message_count',
    color='grade',
    color_discrete_map={
        'A': COLORS['success'],
        'B': COLORS['primary'],
        'C': COLORS['warning'],
        'D': COLORS['secondary'],
        'F': COLORS['danger']
    },
    hover_name='title',
    hover_data=['message_count', 'grade'],
    size_max=50,
    animation_frame='session_num',
    range_x=[0, len(animated_df) + 1],
    range_y=[0, animated_df['message_count'].max() + 10]
)

fig_animated.update_layout(
    title={
        'text': '🚀 Session Journey (Animated)',
        'font': {'size': 24, 'color': COLORS['text']},
        'x': 0.5
    },
    xaxis=dict(
        title='Session Number',
        gridcolor='rgba(255,255,255,0.1)'
    ),
    yaxis=dict(
        title='Messages',
        gridcolor='rgba(255,255,255,0.1)'
    ),
    paper_bgcolor=COLORS['background'],
    plot_bgcolor=COLORS['background'],
    height=500,
    legend=dict(
        title='Grade',
        font=dict(color=COLORS['text']),
        bgcolor='rgba(0,0,0,0.5)'
    )
)

fig_animated.show()

---

## 📊 Executive Summary Dashboard

In [15]:
# === COMBINED DASHBOARD ===

fig_dashboard = make_subplots(
    rows=2, cols=2,
    specs=[
        [{'type': 'indicator'}, {'type': 'pie'}],
        [{'type': 'bar'}, {'type': 'scatter'}]
    ],
    subplot_titles=['Total Sessions', 'Grade Distribution', 'Top 5 Sessions', 'Size vs Count'],
    vertical_spacing=0.15,
    horizontal_spacing=0.1
)

# Indicator
fig_dashboard.add_trace(
    go.Indicator(
        mode="number+delta",
        value=total_sessions,
        delta={'reference': 20, 'relative': True},
        number={'font': {'size': 48, 'color': COLORS['primary']}}
    ),
    row=1, col=1
)

# Pie chart
fig_dashboard.add_trace(
    go.Pie(
        labels=[f"Grade {g}" for g in grade_data.index],
        values=grade_data.values,
        hole=0.4,
        marker=dict(colors=grade_colors[:len(grade_data)]),
        textinfo='percent'
    ),
    row=1, col=2
)

# Bar chart
top5 = sessions_df.head(5)
fig_dashboard.add_trace(
    go.Bar(
        x=top5['title'].str[:15],
        y=top5['message_count'],
        marker=dict(color=COLORS['primary']),
        text=top5['message_count'],
        textposition='outside'
    ),
    row=2, col=1
)

# Scatter
fig_dashboard.add_trace(
    go.Scatter(
        x=list(range(1, len(sessions_df) + 1)),
        y=sessions_df['message_count'],
        mode='markers',
        marker=dict(
            size=10,
            color=sessions_df['message_count'],
            colorscale='Viridis'
        )
    ),
    row=2, col=2
)

fig_dashboard.update_layout(
    title={
        'text': '📈 Executive Summary Dashboard',
        'font': {'size': 28, 'color': COLORS['text']},
        'x': 0.5
    },
    paper_bgcolor=COLORS['background'],
    plot_bgcolor=COLORS['background'],
    height=700,
    showlegend=False,
    font=dict(color=COLORS['text'])
)

fig_dashboard.show()

---

## 🎉 Report Complete!

This interactive dashboard provides a comprehensive view of your chat log analytics.

### Key Insights:
- **31 sessions** analyzed with **483 messages**
- **4 Grade A sessions** (12.9%) representing deep, high-quality conversations
- **Documentation** is the most common topic keyword
- Average session length: **15.6 messages**

### Next Steps:
1. Implement LLM-as-a-Judge for automated quality grading
2. Track quality trends over time
3. Export visualizations for reports

---

*Generated by AI Coding Manager - Chat Analytics Dashboard*